In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.distance import geodesic
import datetime as dt

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, confusion_matrix, recall_score, precision_score, roc_curve, f1_score
from imblearn.over_sampling import SMOTE 

In [2]:
# Ensuring notebook remains tidy
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Setting our seaborn style and palette
sns.set_style('darkgrid')
sns.set_palette('icefire')

In [4]:
# import cleaned datasets/ original Kaggle datasets 
test = pd.read_csv("../data/test.csv", parse_dates=[1])
weather = pd.read_csv("../data/weather_tidied.csv", parse_dates=[1])

In [5]:
print(test.shape)
test.columns = test.columns.str.lower()
test.info()

(116293, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116293 entries, 0 to 116292
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   id                      116293 non-null  int64         
 1   date                    116293 non-null  datetime64[ns]
 2   address                 116293 non-null  object        
 3   species                 116293 non-null  object        
 4   block                   116293 non-null  int64         
 5   street                  116293 non-null  object        
 6   trap                    116293 non-null  object        
 7   addressnumberandstreet  116293 non-null  object        
 8   latitude                116293 non-null  float64       
 9   longitude               116293 non-null  float64       
 10  addressaccuracy         116293 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(3), object(5)
memory usage: 9.8+ MB


In [6]:
def initial_clean(df):
    df.drop(columns=['address', 'block', 'street', 'addressaccuracy'], inplace=True)
    return df

In [7]:
test_iter1 = initial_clean(test)
test_iter1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116293 entries, 0 to 116292
Data columns (total 7 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   id                      116293 non-null  int64         
 1   date                    116293 non-null  datetime64[ns]
 2   species                 116293 non-null  object        
 3   trap                    116293 non-null  object        
 4   addressnumberandstreet  116293 non-null  object        
 5   latitude                116293 non-null  float64       
 6   longitude               116293 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 6.2+ MB


In [8]:
def date_feature_engineering(df):
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['week'] = df['date'].dt.isocalendar().week
    df['year_month'] = df['date'].dt.strftime("%Y-%m")

    return df

In [9]:
test_iter2 = date_feature_engineering(test_iter1)
test_iter2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116293 entries, 0 to 116292
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   id                      116293 non-null  int64         
 1   date                    116293 non-null  datetime64[ns]
 2   species                 116293 non-null  object        
 3   trap                    116293 non-null  object        
 4   addressnumberandstreet  116293 non-null  object        
 5   latitude                116293 non-null  float64       
 6   longitude               116293 non-null  float64       
 7   year                    116293 non-null  int64         
 8   month                   116293 non-null  int64         
 9   week                    116293 non-null  UInt32        
 10  year_month              116293 non-null  object        
dtypes: UInt32(1), datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 9.4+ MB


In [10]:
def dummify_species(df):
    species_of_interest = ['RESTUANS', 'PIPIENS']
    # create loop to create dummified variables to indicate presence of Pipiens & Restuans species
    for i in species_of_interest:
        df[i] = 0
        for index, row in df.iterrows():
            if i in row["species"]:
                df[i][index] = 1
    
    return df

In [11]:
test_iter3 = dummify_species(test_iter2)
test_iter3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116293 entries, 0 to 116292
Data columns (total 13 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   id                      116293 non-null  int64         
 1   date                    116293 non-null  datetime64[ns]
 2   species                 116293 non-null  object        
 3   trap                    116293 non-null  object        
 4   addressnumberandstreet  116293 non-null  object        
 5   latitude                116293 non-null  float64       
 6   longitude               116293 non-null  float64       
 7   year                    116293 non-null  int64         
 8   month                   116293 non-null  int64         
 9   week                    116293 non-null  UInt32        
 10  year_month              116293 non-null  object        
 11  RESTUANS                116293 non-null  int64         
 12  PIPIENS                 116293

In [12]:
# Read wnv risk buckets from folder
wnv_levels = pd.read_csv('../data/wnv_levels.csv')

In [13]:
def dummify_risk_buckets(df, wnv_levels):
    merged_df = df.merge(wnv_levels[['addressnumberandstreet', 'risk_bucket']], how='left', on='addressnumberandstreet')
    merged_dum_df = pd.get_dummies(merged_df, columns=['risk_bucket'], prefix='bucket')
    
    return merged_dum_df

In [14]:
test_iter4 = dummify_risk_buckets(test_iter3, wnv_levels)
test_iter4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116293 entries, 0 to 116292
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   id                      116293 non-null  int64         
 1   date                    116293 non-null  datetime64[ns]
 2   species                 116293 non-null  object        
 3   trap                    116293 non-null  object        
 4   addressnumberandstreet  116293 non-null  object        
 5   latitude                116293 non-null  float64       
 6   longitude               116293 non-null  float64       
 7   year                    116293 non-null  int64         
 8   month                   116293 non-null  int64         
 9   week                    116293 non-null  UInt32        
 10  year_month              116293 non-null  object        
 11  RESTUANS                116293 non-null  int64         
 12  PIPIENS                 116293

In [15]:
def determine_station(lat, long):
    station1 = (41.995, -87.933) #Chicago O'Hare Tntl Airpot -> lat: 41.995 long: -87.933
    station2 = (41.786, -87.752) #Chicago Midway Tntl Airpot -> lat: 41.786 long: -87.752
    coordinates = (lat, long)
    
    return 1 if geodesic(coordinates, station1) < geodesic(coordinates, station2) else 2

In [16]:
%%time

# Apply function to map our function
test_iter4['station_ref'] = test_iter4.apply(lambda i: determine_station(i['latitude'], i['longitude']), axis=1)
test_iter4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116293 entries, 0 to 116292
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   id                      116293 non-null  int64         
 1   date                    116293 non-null  datetime64[ns]
 2   species                 116293 non-null  object        
 3   trap                    116293 non-null  object        
 4   addressnumberandstreet  116293 non-null  object        
 5   latitude                116293 non-null  float64       
 6   longitude               116293 non-null  float64       
 7   year                    116293 non-null  int64         
 8   month                   116293 non-null  int64         
 9   week                    116293 non-null  UInt32        
 10  year_month              116293 non-null  object        
 11  RESTUANS                116293 non-null  int64         
 12  PIPIENS                 116293

In [17]:
# Import tidied weather dataset
weather = pd.read_csv('../data/weather_tidied.csv', parse_dates=[1])
print(weather.shape)
weather.head()

(2944, 13)


,station,date,tmax,tmin,tavg,dewpoint,preciptotal,stnpressure,sealevel,avgspeed,raining,misty,humidity
0,1,2007-05-01,83,50,67.0,51,0.0,29.10,29.82,9.2,0,0,56.410238
1,2,2007-05-01,84,52,68.0,51,0.0,29.18,29.82,9.6,0,0,54.497276
2,1,2007-05-02,59,42,51.0,42,0.0,29.38,30.09,13.4,0,1,71.176939
3,2,2007-05-02,60,43,52.0,42,0.0,29.44,30.08,13.4,0,1,68.594699
4,1,2007-05-03,66,46,56.0,40,0.0,29.39,30.12,11.9,0,0,54.848860


In [18]:
test_iter5 = test_iter4.merge(weather, how='left', left_on=['station_ref', 'date'], right_on=['station', 'date'])
test_iter5.drop(columns=["station"], inplace=True)
test_iter5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116293 entries, 0 to 116292
Data columns (total 29 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   id                      116293 non-null  int64         
 1   date                    116293 non-null  datetime64[ns]
 2   species                 116293 non-null  object        
 3   trap                    116293 non-null  object        
 4   addressnumberandstreet  116293 non-null  object        
 5   latitude                116293 non-null  float64       
 6   longitude               116293 non-null  float64       
 7   year                    116293 non-null  int64         
 8   month                   116293 non-null  int64         
 9   week                    116293 non-null  UInt32        
 10  year_month              116293 non-null  object        
 11  RESTUANS                116293 non-null  int64         
 12  PIPIENS                 116293

In [19]:
def create_pivot(df):
    # Aggregating weather data by year and month
    weather['year_month'] = weather['date'].dt.strftime('%Y-%m')
    
    # Adding this to use as key for lag variables
    df['year_month_lag1'] = (df['date'] - pd.DateOffset(months=1)).dt.strftime('%Y-%m')

    # Creating a temporary pivot table to extract weather states by year month
    pivot = pd.DataFrame(pd.pivot_table(data=weather, index=['station','year_month'], 
                                        values=['tmin', 'tmax', 'tavg', 'dewpoint', 'preciptotal', 'sealevel', 
                                                'avgspeed', 'humidity'], aggfunc='mean').to_records())
    
    old_col = pivot.drop(columns='station').columns
    new_col = old_col + "_"
    pivot = pivot.rename(columns = dict(zip(old_col.tolist(), new_col.tolist())))
    
    # Assigning aggregated year month values into train_merged_df from pivot
    for i in new_col[1:]:
        print(i)
#         df = pivot[['station', 'year_month_', i]].merge(df, how='right', left_on=['station', 'year_month_'], 
#                                                         right_on=['station_ref', 'year_month']).rename(columns={i: i + 'lag0'}).drop(['station', 'year_month_'], axis='columns')
        
        df = pivot[['station', 'year_month_', i]].merge(df, how='right', left_on=['station', 'year_month_'], 
                                                        right_on=['station_ref', 'year_month_lag1']).rename(columns={i: i + 'lag1'}).drop(['station', 'year_month_'], axis='columns')
    
    return df

In [20]:
test_iter6 = create_pivot(test_iter5)
test_iter6.info()

avgspeed_
dewpoint_
humidity_
preciptotal_
sealevel_
tavg_
tmax_
tmin_
<class 'pandas.core.frame.DataFrame'>
Int64Index: 116293 entries, 0 to 116292
Data columns (total 38 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   tmin_lag1               116293 non-null  float64       
 1   tmax_lag1               116293 non-null  float64       
 2   tavg_lag1               116293 non-null  float64       
 3   sealevel_lag1           116293 non-null  float64       
 4   preciptotal_lag1        116293 non-null  float64       
 5   humidity_lag1           116293 non-null  float64       
 6   dewpoint_lag1           116293 non-null  float64       
 7   avgspeed_lag1           116293 non-null  float64       
 8   id                      116293 non-null  int64         
 9   date                    116293 non-null  datetime64[ns]
 10  species                 116293 non-null  object        
 11  trap                

In [21]:
test_iter6.dropna(inplace=True)
test_iter6.drop(columns=['date', 'station_ref', 'year_month', 
                                'year_month_lag1', 'species', 
                                "addressnumberandstreet", "trap", "longitude", "latitude"], inplace=True)

In [22]:
# generate predictions
pred = predict(test_data)
pred

id                        0
date                      0
species                   0
trap                      0
addressnumberandstreet    0
latitude                  0
longitude                 0
year                      0
month                     0
week                      0
year_month                0
RESTUANS                  0
PIPIENS                   0
dtype: int64